# Getting Started with MakeTables

This notebook contains interactive examples from the README to get you started with MakeTables.

## Installation

First, make sure you have MakeTables installed (once it is published on PyPI):

```bash
pip install maketables
```

For development installation:
```bash
git clone https://github.com/dsliwka/maketables.git
cd maketables
pip install -e .
```

## Descriptive Statistics Table

Let's start with creating a descriptive statistics table using the classic auto dataset (Note that you can work with any pandas DataFrame but MakeTables also supports loading and saving dta files (Stata) as this facilitates working with variable labels).


In [1]:
import pandas as pd
import maketables as mt

# Load your data (here using a sample Stata dataset with the import_dta function that also stores variable labels)
df = mt.import_dta("https://www.stata-press.com/data/r18/auto.dta")

In [2]:
# Create descriptive statistics table
mt.DTable(df, vars=["mpg","weight","length"], bycol=["foreign"])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1fa6c9e2a20>

## Regression Tables

### Using PyFixest
Use the extremely versatile [PyFixest](https://py-econometrics.github.io/pyfixest/pyfixest.html) package.

In [3]:
import pyfixest as pf

# Fit your models here using pyfixest
est1 = pf.feols("mpg ~ weight", data=df)
est2 = pf.feols("mpg ~ weight + length", data=df)
est3 = pf.feols("mpg ~ weight + length | foreign", data=df)

# Make the table
mt.ETable([est1, est2, est3])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1fa14d33b00>

In [4]:
# Test with a full ETable to see if the issue is fixed
import importlib
import maketables
importlib.reload(maketables)

# Recreate the table with the fix
mt.ETable([est1, est2, est3])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1fa6cc33530>

### Using Statsmodels

MakeTables also works with [Statsmodels](https://www.statsmodels.org/stable/index.html) for various types of regression models. Here is for instance a linear probability model and a probit.

In [5]:
import statsmodels.formula.api as smf

# Generate a dummy variable and label it
df["foreign_i"] = (df["foreign"] == "Foreign")*1
mt.set_var_labels(df, {"foreign_i": "Foreign (indicator)"})

# Fit your models 
est1 = smf.ols("foreign_i ~ weight + length + price", data=df).fit()
est2 = smf.probit("foreign_i ~ weight + length + price", data=df).fit(disp=0)

# Make the table
mt.ETable([est1, est2], model_stats=["N","r2","pseudo_r2"], model_heads=["OLS","Probit"])

<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1fa18450a40>

### Combining different packages

You can also combine models from different packages in one table. 

In [6]:
# Here we just reestimaate the linear probability model using pyfixest instead of statsmodels
est0=pf.feols("foreign_i ~ weight + length + price", data=df)

# Make the table
mt.ETable([est0, est1, est2], model_stats=["N","r2","pseudo_r2"], model_heads=["OLS (PyFixest)","OLS (Statsmodels)","Probit"])


<maketables.mtable.MTable.__repr__.<locals>.DualOutput at 0x1fa1b1a7fb0>